In [ ]:
####################################################################################################

#    Copyright 2019 Srijan Verma and EMBL-European Bioinformatics Institute

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

####################################################################################################

In [114]:
import pandas as pd
df_overlap = pd.read_csv('/feature_selection/overlap/Ens-Ref/ens_ref_gene_overlap_with_ov_len_and_no_exons_same_strand.csv', index_col=0)

In [115]:
df_overlap.head(2)

,ENS lincRNA gene ID,Ref lncRNA gene ID,Chr region,No. of transcripts in lincrna,No. of transcripts in lncrna,ENS-Ref Overlap ?,same_opp_overlap,Overlap Length,No. ENS lincrna Exons which overlap RefSeq exons
0,ENSG00000236648,gene-LOC107984028,1,1,4,Yes,1,17270,3
1,ENSG00000230714,gene-LOC105372922,1,1,1,Yes,0,-,-


In [116]:
drop_index = []
drop_no = 0
for i in tqdm(range(len(df_overlap))):
    if df_overlap['same_opp_overlap'][i] == 0:
        drop_index.append(i)
        drop_no = drop_no + 1

100%|██████████| 6252/6252 [00:00<00:00, 39034.50it/s]


In [117]:
df_overlap.drop(drop_index, axis=0,inplace=True)
df_overlap.reset_index(drop=True, inplace=True)

In [118]:
df_overlap.head(2)

,ENS lincRNA gene ID,Ref lncRNA gene ID,Chr region,No. of transcripts in lincrna,No. of transcripts in lncrna,ENS-Ref Overlap ?,same_opp_overlap,Overlap Length,No. ENS lincrna Exons which overlap RefSeq exons
0,ENSG00000236648,gene-LOC107984028,1,1,4,Yes,1,17270,3
1,ENSG00000230704,gene-LOC101928628,1,1,1,Yes,1,1720,2


In [119]:
df_overlap.to_csv('/feature_selection/overlap/Ens-Ref/ens_ref_gene_overlap_cleaned.csv')

In [120]:
from tqdm import tqdm
l_id = []
r_id = []
for i in tqdm(range(len(df_overlap['ENS lincRNA gene ID']))):
    l_id.append(df_overlap['ENS lincRNA gene ID'][i])
    r_id.append(df_overlap['Ref lncRNA gene ID'][i])

100%|██████████| 4557/4557 [00:00<00:00, 18110.64it/s]


In [121]:
set_lincrna_id = set(l_id)
set_ref_id = set(r_id)

In [122]:
df_ens_transcript_data = pd.read_csv('/feature_selection/ORF/Ensembl/all_lincrna_transcript_data_orf_info.csv', index_col=0)

In [123]:
drop_index = []
drop_no = 0
for i in tqdm(range(len(df_ens_transcript_data))):
    if df_ens_transcript_data['Len of longest ORF'][i] == 'No start codon found':
        drop_index.append(i)
        drop_no = drop_no + 1

100%|██████████| 13666/13666 [00:00<00:00, 37216.98it/s]


In [124]:
df_ens_transcript_data.drop(drop_index, axis=0,inplace=True)
df_ens_transcript_data.reset_index(drop=True, inplace=True)

In [125]:
len(df_ens_transcript_data)

13637

In [126]:
df_ens_transcript_data["Len of longest ORF"] = pd.to_numeric(df_ens_transcript_data["Len of longest ORF"])

In [127]:
df_ens_transcript_data.head(3)

,SNO,Gene ID,Transcript ID,Biotype,Strand,Seq region Name,Transcript Start,Transcript End,Transcript Length,No. of Exons,...,Match ?,Sequences,GC %,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,4916,ENSG00000236423,ENST00000413332,lincRNA,1,1,3900372,3905532,634,3,...,YES,GCGAAGGTCACAGGCCTTGGGCCAGTGGTGGGACACTGGAGCAGGA...,59.31,33,11,ATGTTCATGAGGGCGCGTCCCTTGGCACCCCTGTAG,65,MFMRARPLAPL,YES,YES
1,11412,ENSG00000270035,ENST00000602973,lincRNA,-1,1,7698303,7698872,570,1,...,YES,GGCACAGGGGAGGCCCTGAGTCACACGATGGTGATGCCCAGGGCAG...,55.26,42,14,ATGGTGATGCCCAGGGCAGAAGATGTCAGAAGACAGGCAGTTTGA,27,MVMPRAEDVRRQAV,YES,YES
2,11417,ENSG00000270066,ENST00000602755,lincRNA,1,1,109100193,109100619,427,1,...,YES,GTTTTAGGGAGGGAGAGCGGCCTGGGTCCTGGGTGTTGTGTGCGGA...,65.81,138,46,ATGAAAAGCTCTCCTCCTGCGAAGCCCCCTCGGGGCGCTGTGCAGC...,227,MKSSPPAKPPRGAVQRGPLGGGHAVVPEVPELALRGPALRSDELIR,YES,YES


In [140]:
cleaned_max_orf = []
cleaned_amino_acid = []
cleaned_max_index = []
cleaned_gc_percentage = []
cleaned_max_orf_seq = []

In [141]:
def get_gene_orf():
    for i in tqdm(range(len(df_overlap))):
        a = df_ens_transcript_data.index[df_ens_transcript_data['Gene ID'] == df_overlap['ENS lincRNA gene ID'][i]]
        max_orf = 0
        max_gc = 0
        index_max_all = 0
        for i in range(len(a)):
            if df_ens_transcript_data.iloc[a[i]]['Len of longest ORF'] > max_orf:
                max_orf = df_ens_transcript_data.iloc[a[i]]['Len of longest ORF']
                max_gc = df_ens_transcript_data.iloc[a[i]]['GC %']
                index_max_all = a[i]
            elif df_ens_transcript_data.iloc[a[i]]['Len of longest ORF'] == max_orf:
                if df_ens_transcript_data.iloc[a[i]]['GC %'] > max_gc:
                    max_gc = df_ens_transcript_data.iloc[a[i]]['GC %']
                    index_max_all = a[i]
        
        cleaned_max_index.append(index_max_all)
        cleaned_max_orf.append(max_orf)
        cleaned_amino_acid.append(df_ens_transcript_data.iloc[index_max_all]['No. of Amino Acids'])
        cleaned_gc_percentage.append(max_gc)
        cleaned_max_orf_seq.append(df_ens_transcript_data.iloc[index_max_all]['Longest ORF Seq'])

In [142]:
%%time
get_gene_orf()

100%|██████████| 4557/4557 [01:09<00:00, 66.04it/s] 

CPU times: user 48.9 s, sys: 1.78 s, total: 50.7 s
Wall time: 1min 9s


In [147]:
print(cleaned_max_orf[5])
print(cleaned_gc_percentage[5])
print(cleaned_amino_acid[5])

201
43.26
67


In [157]:
df_overlap['Max ORF length(ENS)'] = cleaned_max_orf
df_overlap['Max No. of amino acids(ENS)'] = cleaned_amino_acid
df_overlap['Max Transcript Index for ORF(ENS)'] = cleaned_max_index
df_overlap['GC % of trans row having Max ORF(ENS)'] = cleaned_gc_percentage
df_overlap['Max ORF Sequence(ENS)'] = cleaned_max_orf_seq

In [159]:
df_overlap.to_csv('/feature_selection/ORF/Ens-Ref/ens_ref_ORF&overlap_cleaned(ens).csv')